In [1]:
import json

import pandas as pd
import numpy as np
import os

from hydra import compose, initialize
from omegaconf import OmegaConf
from sklearn.metrics import accuracy_score, balanced_accuracy_score
from tqdm import tqdm
from PIL import Image
import torch
from pathlib import Path
from PIL import ImageFile

from closedset_model import build_model
from competition_metrics import evaluate
from datasets import get_valid_transform
from paths import METADATA_DIR, VAL_DATA_DIR
from utils import copy_config, get_device

np.set_printoptions(precision=5)
ImageFile.LOAD_TRUNCATED_IMAGES = True

class PytorchWorker:
    """Run inference using PyTorch."""

    def __init__(self, model_path: str, number_of_categories: int = 1784, model_id="efficientnet_b0", device="cpu", transforms=None):

        ########################################
        # must be set before calling _load_model
        self.number_of_categories = number_of_categories
        self.model_id = model_id
        self.device = device
        ########################################

        self.transforms = transforms
        # most other attributes must be set before calling _load_model, so call last
        self.model = self._load_model(model_path)

    def _load_model(self, model_path):
        print("Setting up Pytorch Model")
        # model = models.efficientnet_b0()
        # model.classifier[1] = nn.Linear(in_features=1280, out_features=self.number_of_categories)
        model = build_model(
            model_id=self.model_id,
            pretrained=False,
            fine_tune=False,
            num_classes=self.number_of_categories,
            # this is all that matters. everything else will be overwritten by checkpoint state
            dropout_rate=0.5,
        ).to(self.device)
        model_ckpt = torch.load(model_path, map_location=self.device)
        model.load_state_dict(model_ckpt['model_state_dict'])

        return model.to(self.device).eval()

    def predict_image(self, image: np.ndarray) -> list():
        """Run inference using ONNX runtime.

        :param image: Input image as numpy array.
        :return: A list with logits and confidences.
        """

        img = self.transforms(image)
        
        if isinstance(img, tuple):
            img = torch.cat([instance.unsqueeze(0) for instance in img])
            img = torch.unique(img, dim=0)

        if img.dim() < 4:
            img = img.unsqueeze(0)
        
        img = img.to(self.device)
        
        logits = self.model(img)

        return logits


def get_probas(test_metadata, model_id, model_path, images_root_path, device, transforms):
    """Make submission file"""

    model = PytorchWorker(model_path, model_id=model_id, device=device, transforms=transforms)

    probas_total = []
    image_paths = test_metadata["image_path"]

    with torch.no_grad():
        for image_path in tqdm(image_paths):
            image_path = os.path.join(images_root_path, image_path)
            test_image = Image.open(image_path).convert("RGB")
            logits = model.predict_image(test_image)
            probas = torch.nn.functional.softmax(logits, dim=-1).cpu().numpy()
            if probas.shape[0] > 1:
                probas = np.mean(probas, axis=0)
            probas = probas.squeeze()
            probas_total.append(probas)
    
    return probas_total


def evaluate_experiment(cfgs, trial_name=None, multi_instance=False, device="cpu", multicrop=False, debug=False):

    submission_file_path = "test-time-augmengations-submission.csv"
    if trial_name is not None:
        submission_file_path = trial_name + submission_file_path

    metadata_file_path = METADATA_DIR / "SnakeCLEF2023-ValMetadata.csv"
    test_metadata = pd.read_csv(metadata_file_path)
    if debug:
        test_metadata = test_metadata.head(20)
    if not multi_instance:
        test_metadata.drop_duplicates("observation_id", keep="first", inplace=True)
    
    probas_per_model = []
    for cfg in cfgs:
        experiment_id = cfg["experiment_id"]
        if debug: print(f"getting probas for experiment {experiment_id}")
        model_id = cfg["model_id"]
        image_size = cfg["image_size"]
        transforms = get_valid_transform(image_size=image_size, pretrained=True, fivecrop=multicrop)
        experiment_dir = Path("model_checkpoints") / experiment_id
        predictions_output_csv_path = str(experiment_dir / "submission.csv")
        model_file = "model.pth"
        model_path = str(experiment_dir / model_file)
        probas = get_probas(
            model_id=model_id,
            test_metadata=test_metadata,
            model_path=model_path,
            images_root_path=VAL_DATA_DIR,
            device=device,
            transforms=transforms,
        )
        probas_per_model.append(probas)
    probas_per_model = np.array(probas_per_model)
    if debug: print("probas_per_model.shape", probas_per_model.shape)
    if len(cfgs) > 1:
        averaged_probas = np.mean(probas_per_model, axis=0)
    else:
        averaged_probas = probas_per_model.squeeze()
    if debug: print("averaged_probas.shape", averaged_probas.shape)
    # if debug: print("np.argmax(averaged_probas)", np.argmax(averaged_probas))

    if multi_instance:
        preds = []
        # pandas unique preserves order
        for obs_id in test_metadata["observation_id"].unique():
            indices = list(test_metadata["observation_id"].loc[lambda x: x==obs_id].index)
            if len(indices) > 1:
                if debug: print("indices", indices)
                observation_probas = averaged_probas[indices, :]
                observation_average = np.mean(averaged_probas[indices], axis=0)
                if debug: print("observation_average.shape", observation_average.shape)
                preds.extend([np.argmax(observation_average)] * len(indices))
            else:
                preds.append(np.argmax(averaged_probas[indices], axis=1)[0])
    else:
        preds = np.argmax(averaged_probas, axis=1)

    if debug:
        print("preds", preds)
        if isinstance(preds, list):
            preds = np.array(preds)
        print("preds.shape", preds.shape)
    
    submission_df = test_metadata.copy()
    submission_df["class_id"] = preds
    submission_df = submission_df[["observation_id", "class_id"]]
    submission_df.drop_duplicates("observation_id", keep="first", inplace=True)
    submission_df.to_csv(submission_file_path, index=False)

    competition_metrics_scores = evaluate(
        test_annotation_file=metadata_file_path,
        user_submission_file=submission_file_path,
        phase_codename="prediction-based",
    )["submission_result"]

    return competition_metrics_scores

/root/.cache/pypoetry/virtualenvs/snakeclef-jsOUoRFY-py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
performances = dict()
start = 1
stop = 2
step = .1
for mult in np.arange(start, stop + step, step):
    res = int(mult*384)
    cfgs = [
    {"model_id": "caformer_s18.sail_in22k_ft_in1k_384",
     "experiment_id": "2024-05-05 22:41:41.115323",
    "image_size": res,},
    ]
    device = get_device()
    scores = evaluate_experiment(cfgs=cfgs, trial_name="ensemble_multi-instance",
                                                multi_instance=False, multicrop=False, device=device, debug=False)
    performances[res] = scores

print(performances)

Using device: cuda:0
Setting up Pytorch Model
Not loading pre-trained weights
Freezing hidden layers...


100%|██████████| 7816/7816 [07:58<00:00, 16.33it/s]


Starting Evaluation.....
Evaluating for Prediction-based Phase
Evaluated scores: {'F1 Score': 48.97, 'Accuracy': 65.08, 'PSC': (32.99, 3.79, 9.16, 18.0), 'PSC_total': (2078, 239, 139, 273), 'Track1 Metric': 84.24, 'Track2 Metric': 3797}
Completed evaluation
Using device: cuda:0
Setting up Pytorch Model
Not loading pre-trained weights
Freezing hidden layers...


100%|██████████| 7816/7816 [07:52<00:00, 16.55it/s]


Starting Evaluation.....
Evaluating for Prediction-based Phase
Evaluated scores: {'F1 Score': 50.86, 'Accuracy': 66.88, 'PSC': (30.74, 3.89, 8.5, 18.39), 'PSC_total': (1936, 245, 129, 279), 'Track1 Metric': 84.82, 'Track2 Metric': 3629}
Completed evaluation
Using device: cuda:0
Setting up Pytorch Model
Not loading pre-trained weights
Freezing hidden layers...


...

100%|██████████| 7816/7816 [08:20<00:00, 15.60it/s]


Starting Evaluation.....
Evaluating for Prediction-based Phase
Evaluated scores: {'F1 Score': 53.14, 'Accuracy': 69.92, 'PSC': (27.94, 3.3, 6.39, 18.85), 'PSC_total': (1760, 208, 97, 286), 'Track1 Metric': 86.27, 'Track2 Metric': 3233}
Completed evaluation
Using device: cuda:0
Setting up Pytorch Model
Not loading pre-trained weights
Freezing hidden layers...


100%|██████████| 7816/7816 [09:32<00:00, 13.65it/s]


Starting Evaluation.....
Evaluating for Prediction-based Phase
Evaluated scores: {'F1 Score': 53.22, 'Accuracy': 70.43, 'PSC': (27.72, 3.06, 6.66, 17.86), 'PSC_total': (1746, 193, 101, 271), 'Track1 Metric': 86.4, 'Track2 Metric': 3179}
Completed evaluation
Using device: cuda:0
Setting up Pytorch Model
Not loading pre-trained weights
Freezing hidden layers...


 64%|██████▍   | 4991/7816 [05:34<03:44, 12.59it/s]

In [9]:
dfs = []
for res, perf in performances.items():
    df = pd.DataFrame({"res": [res], "Track 1": [perf["Track1 Metric"]]})
    dfs.append(df)
comparison_df = pd.concat(dfs, ignore_index=True)

In [10]:
comparison_df.sort_values("Track 1", ascending=False)

,res,Track 1
5,576,86.77
7,652,86.65
6,614,86.51
9,729,86.40
4,537,86.36
10,768,86.33
8,691,86.27
3,499,85.99
2,460,85.79
1,422,84.82


In [11]:
comparison_df.sort_values("res", ascending=False)

,res,Track 1
10,768,86.33
9,729,86.40
8,691,86.27
7,652,86.65
6,614,86.51
5,576,86.77
4,537,86.36
3,499,85.99
2,460,85.79
1,422,84.82
